In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5,4'

In [2]:
from jax import lax, random, numpy as jnp
import jax
import haiku as hk

In [3]:
jax.local_devices()

[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=1, process_index=0, slice_index=0)]

In [4]:
# A helper function to randomly initialize weights and biases
# for a dense neural network layer
def forward(x):
    return hk.nets.MLP([300,3000, 10])(x)

In [5]:
model = hk.transform(forward)
params = model.init(random.PRNGKey(42), jnp.ones((10)))

In [6]:
hk.data_structures.tree_size(params)

936310

In [6]:
params["mlp/~/linear_0"]["w"].device()

StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)

In [7]:
apply_1 = jax.pmap(lambda params, inputs: model.apply(params, jax.random.PRNGKey(0), inputs), in_axes=(None, 0))

In [13]:
%timeit apply_1(params, jnp.ones((2,10)))

1.67 ms ± 83.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
apply_2 = jax.pmap(lambda params, inputs: model.apply(params, jax.random.PRNGKey(0), inputs))

In [14]:

params_repl = jax.device_put_replicated(params, jax.local_devices())
%%timeit 
apply_2(params_repl, jnp.ones((2,10)))

NameError: name 'params_repl' is not defined

In [11]:
batch_repl = jax.device_put_sharded(jnp.ones((2,10)), jax.local_devices())

ValueError: device_put_sharded `shards` input must be a sequence; got <class 'jaxlib.xla_extension.DeviceArray'>

In [ ]:
%%timeit 
params_repl = [params]*2
params_repl = jax.device_put_sharded(params_repl, jax.local_devices())
apply(params_repl, jnp.ones((2,10)))

2.21 ms ± 90.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
params_repl = [params]*2
params_repl = jax.device_put_sharded(params_repl, jax.local_devices())

In [ ]:
params_repl = jax.device_put_replicated(params, jax.local_devices())

In [ ]:
params_repl["mlp/~/linear_0"]["b"][0].device()

StreamExecutorGpuDevice(id=1, process_index=0, slice_index=0)